# <center>[🦏 Автоматический стэкинг и блендинг](https://stepik.org/lesson/872530/)</center>

<div class="alert alert-info">

Как вы уже могли заметить, чем больше моделей вы стэкаете, тем больше у вас разрастается код, а кол-во беспорядка в нем растет по экспоненте. Но есть специальные инструменты, которые позволяют сделать это элегантно и даже более эффективно. Да еще и меньшим числом строк кода! 
    

<div class="alert alert-info">
    
Сегодня мы поговорим про `sklearn.Pipelines` - способ упаковать ваш процесс обучения и инференса от `Feature Engineering`а до стэкинга 10 моделей в один пайплайн.

### Оглавление ноутбука

<img src='../images/pipelines.jpg' align="right" width="600" height="600" />
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#c1">🛠 Три модели для блендинга </a></li>
<li><a href="#look1">🔧 Построение пайплана</a>
<li><a href="#check1"> 🔋 Принципы блендинга</a>
<li><a href="#6">🧸 Выводы и заключения</a>

</li></ul></font></p>


## Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import lightgbm
import xgboost
import catboost

In [2]:
# pip install xgboost -U -q

In [3]:
import warnings
warnings.filterwarnings("ignore")

## Считываем данные

In [4]:
from sklearn import preprocessing
data = pd.read_csv('../data/car_train.csv')

categorical_features = ['model', 'car_type', 'fuel_type']

for cat in categorical_features:
    lbl = preprocessing.LabelEncoder()
    data[cat] = lbl.fit_transform(data[cat].astype(str))
    data[cat] = data[cat].astype('category')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   car_id         2337 non-null   object  
 1   model          2337 non-null   category
 2   car_type       2337 non-null   category
 3   fuel_type      2337 non-null   category
 4   car_rating     2337 non-null   float64 
 5   year_to_start  2337 non-null   int64   
 6   riders         2337 non-null   int64   
 7   year_to_work   2337 non-null   int64   
 8   target_reg     2337 non-null   float64 
 9   target_class   2337 non-null   object  
dtypes: category(3), float64(2), int64(3), object(2)
memory usage: 136.3+ KB


In [5]:
data['car_id'].nunique(), data.shape

(2337, (2337, 10))

### Разделим выборку на валидационную и обучающую

In [6]:
# значения таргета закодируем целыми числами
class_names = np.unique(data['target_class'])
data['target_class'] = data['target_class'].replace(class_names, np.arange(data['target_class'].nunique()))

In [7]:
cols2drop = ['car_id', 'target_reg', 'target_class']

In [8]:
X_train, X_val, y_train, y_val = train_test_split(data.drop(cols2drop, axis=1), 
                                                    data['target_class'],
                                                    test_size=.25,
                                                    stratify=data['target_class'],
                                                    random_state=42)
print(X_train.shape, X_val.shape)

(1752, 7) (585, 7)


# <center> Объявим 3 модели

### Модель `CatBoost`

In [9]:
params_cat = {
             'n_estimators' : 1000,
              # 'learning_rate': .03,
              'depth' : 3,
              'verbose': False,
              'use_best_model': True,
              'cat_features' : categorical_features,
              'text_features': [],
              # 'train_dir' : '/home/jovyan/work/catboost',
              'border_count' : 64,
              'l2_leaf_reg' : 1,
              'bagging_temperature' : 2,
              'rsm' : 0.1,
              'loss_function': 'MultiClass',
              'auto_class_weights' : 'Balanced', #try not balanced
              'random_state': 42,
              'use_best_model': False,
              # 'custom_metric' : ['AUC', 'MAP'] # Не работает внутри Sklearn.Pipelines
         }

In [10]:
cat_model = catboost.CatBoostClassifier(**params_cat)

### Модель `LightGBM`

In [33]:
categorical_features_index = [i for i in range(data.shape[1]) if data.columns[i] in categorical_features]
params_lgbm = {'num_leaves': 887,
               'n_estimators': 480,
               'max_depth': 7,
               'min_child_samples': 1073,
               'learning_rate': 0.053,
               'min_data_in_leaf': 2,
               'feature_fraction': 0.95,
               'categorical_feature': categorical_features_index
              }

In [35]:
lgbm_model = lightgbm.LGBMClassifier(**params_lgbm)

### Модель `XGBoost`

In [36]:
params_xgb = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    # 'colsample_bytree': 0.7, # Нельзя одновременно с subsample
    'objective': 'reg:linear',
    # 'eval_metric': 'accuracy'
    # 'enable_categorical' : True
}

In [37]:
xgb_model = xgboost.XGBClassifier(**params_xgb)

# <center> Построим пайплан

In [38]:
# Вспомогательные блоки организации для пайплайна
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

# Вспомогательные элементы для наполнения пайплайна
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder, MinMaxScaler

# Некоторые модели для построение ансамбля
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

# Добавим визуализации
import sklearn
sklearn.set_config(display='diagram')

### Предобработаем данные
Под каждый тип данных заводим свой трансформер

In [39]:
categorical_features = ['model', 'car_type', 'fuel_type']
numerical_features = ['car_rating', 'year_to_start', 'riders', 'year_to_work']

In [40]:
# заменяет пропуски самым частым значением и делает ohe
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy='most_frequent')),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))])

In [41]:
# заменяет пропуски средним значением и делает номрализацию
numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer()),
    ("scaler", StandardScaler())
])

In [42]:
# соединим два предыдущих трансформера в один
preprocessor = ColumnTransformer(transformers=[
    ("numerical", numerical_transformer, numerical_features),
    ("categorical", categorical_transformer, categorical_features)])

preprocessor

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['car_rating', 'year_to_start', 'riders',
                                  'year_to_work']),
                                ('categorical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['model', 'car_type', 'fuel_type'])])

In [43]:
preprocessor.transformers[0]

('numerical',
 Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())]),
 ['car_rating', 'year_to_start', 'riders', 'year_to_work'])

# <center> 🎓🐊 Обучим ансамбль

In [44]:
# список базовых моделей
estimators = [
    # ("svm", make_pipeline(preprocessor, LinearSVC(verbose=False))),
    ("random_forest",  make_pipeline(preprocessor, RandomForestClassifier(n_jobs=-1, verbose=False))),
    ("xgboost",  make_pipeline(preprocessor, xgb_model)),
    
    # ("xgboost", xgb_model),
    ("lightgbm", lgbm_model),
    ("catboost", cat_model),
]

# в качестве мета-модели будем использовать LogisticRegression
meta_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(verbose=False),
    n_jobs=-1,
    verbose=False,
)

stacking_classifier = meta_model
stacking_classifier

StackingClassifier(estimators=[('random_forest',
                                Pipeline(steps=[('columntransformer',
                                                 ColumnTransformer(transformers=[('numerical',
                                                                                  Pipeline(steps=[('imputer',
                                                                                                   SimpleImputer()),
                                                                                                  ('scaler',
                                                                                                   StandardScaler())]),
                                                                                  ['car_rating',
                                                                                   'year_to_start',
                                                                                   'riders',
                                                                                   'year_to_work']),
                                                                                 ('categorical',
                                                                                  Pipeline(steps=[('imputer',
                                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                                  ('oneho...
                                                               objective='reg:linear', ...))])),
                               ('lightgbm',
                                LGBMClassifier(categorical_feature=[1, 2, 3],
                                               feature_fraction=0.95,
                                               learning_rate=0.053, max_depth=7,
                                               min_child_samples=1073,
                                               min_data_in_leaf=2,
                                               n_estimators=480,
                                               num_leaves=887)),
                               ('catboost',
                                <catboost.core.CatBoostClassifier object at 0x7f4e5ee82040>)],
                   final_estimator=LogisticRegression(verbose=False), n_jobs=-1,
                   verbose=False)

In [45]:
stacking_classifier.fit(X_train, y_train)

/home/jovyan/.local/lib/python3.9/site-packages/sklearn/impute/_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)
/home/jovyan/.local/lib/python3.9/site-packages/sklearn/impute/_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warnin

[LightGBM] [Warning] feature_fraction is set=0.95, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.95
[LightGBM] [Warning] min_data_in_leaf is set=2, min_child_samples=1073 will be ignored. Current value: min_data_in_leaf=2
[LightGBM] [Warning] feature_fraction is set=0.95, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.95
[LightGBM] [Warning] min_data_in_leaf is set=2, min_child_samples=1073 will be ignored. Current value: min_data_in_leaf=2
[LightGBM] [Warning] feature_fraction is set=0.95, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.95
[LightGBM] [Warning] min_data_in_leaf is set=2, min_child_samples=1073 will be ignored. Current value: min_data_in_leaf=2
[LightGBM] [Warning] feature_fraction is set=0.95, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.95
[LightGBM] [Warning] min_data_in_leaf is set=2, min_child_samples=1073 will be ignored. Current value: min_data_in_leaf=2
[LightGB

StackingClassifier(estimators=[('random_forest',
                                Pipeline(steps=[('columntransformer',
                                                 ColumnTransformer(transformers=[('numerical',
                                                                                  Pipeline(steps=[('imputer',
                                                                                                   SimpleImputer()),
                                                                                                  ('scaler',
                                                                                                   StandardScaler())]),
                                                                                  ['car_rating',
                                                                                   'year_to_start',
                                                                                   'riders',
                                                                                   'year_to_work']),
                                                                                 ('categorical',
                                                                                  Pipeline(steps=[('imputer',
                                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                                  ('oneho...
                                                               objective='reg:linear', ...))])),
                               ('lightgbm',
                                LGBMClassifier(categorical_feature=[1, 2, 3],
                                               feature_fraction=0.95,
                                               learning_rate=0.053, max_depth=7,
                                               min_child_samples=1073,
                                               min_data_in_leaf=2,
                                               n_estimators=480,
                                               num_leaves=887)),
                               ('catboost',
                                <catboost.core.CatBoostClassifier object at 0x7f4e5ee82040>)],
                   final_estimator=LogisticRegression(verbose=False), n_jobs=-1,
                   verbose=False)

In [23]:
# for i in stacking_classifier.estimators:
#     print(i[0])

In [24]:
# dir(stacking_classifier)

In [46]:
for model, (name, _) in zip(stacking_classifier.estimators_, stacking_classifier.estimators):
    preprocessed = stacking_classifier.estimators[0][1].steps[0][1].fit(X_train, y_train).transform(X_val)
    print(name, 'accuracy: ', round(accuracy_score(model.predict(X_val), y_val), 4))

random_forest accuracy:  0.1145
xgboost accuracy:  0.1179
lightgbm accuracy:  0.1248
catboost accuracy:  0.1128


In [47]:
print('ensemble score:', round(accuracy_score(stacking_classifier.predict(X_val), y_val), 4))
# как вы можете заметить ансамбль довольно заметно улучшил качество решения

ensemble score: 0.1094


# <center> 🧸 Выводы и заключения

<div class="alert alert-info">

`sklearn.Pipelines` это очень сильный инструмент, позволяющий упаковать весь процесс обучения модели в один механизм, в который легко добавлять новые модели и который легко применять на инференсе. Кстати, это один из тех инструментов, который часто используется не только на соревнованиях, но и в обычной работе засчет своей элегантности и простоты.